# Cleaning Henry's Law Constants Dataset

Here, we clean up the csv generated from Tabula and make sure everything is in the right format, and at the end there are 4632 unique species.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('henrys_law_dataset.csv')

In [3]:
df

,0,1,2,3
0,NaN,NaN,Inor,ganic species
1,NaN,NaN,O,xygen (O)
2,NaN,oxygen,1.2×10−5,1700 Warneck and Williams (2012) L
3,NaN,O2,1.3 ×10−5,1500 Sander et al. (2011) L
4,NaN,[7782-44-7],1.3×10−5,1500 Sander et al. (2006) L
...,...,...,...,...
23823,NaN,(methyltriethyl lead),NaN,NaN
23824,NaN,[1762-28-3],NaN,NaN
23825,NaN,tetraethyllead,1.3×10−5,6400 Feldhake and Stevens (1963) M
23826,NaN,C8H20Pb,1.3×10−5,Abraham (1979) ?


In [4]:
# as we can see above the first column contains no data so can be removed, the last column also contains reference data which isn't needed so can also be removed
df = df.drop(df.columns[[0, 3]], axis=1)

In [5]:
# renaming column names
df.rename(columns={'1': 'Substance', '2': 'Hcp'}, inplace=True)

In [6]:
# there are some incomplete names as some names are very long are take up more than one row so we need to correct these
# first we deal with if the name ends in a '-' or ',' or ')' sign then it is incomplete

# make sure all values in Substance are strings so they can be worked with
df['Substance'] = df['Substance'].astype(str)

# first a column containing the substance name shifted up by 1 is created - this matches any parts of incomplete names together
df['name endings'] = df['Substance'].shift(-1)

df['Substance'] = df.apply(lambda r: (r['Substance'] + r['name endings']) if r['Substance'].endswith('-') or r['Substance'].endswith(',') or r['Substance'].endswith(')') else r['Substance'], axis=1)

# the above works well if the name occupies two rows but what if it takes up three rows?
# we need to repeat the process again and shift name endings column up by one so that any leftover of the already combined names can be matched up
df['name endings 2'] = df['Substance'].shift(-2)

df['Substance'] = df.apply(lambda r: (r['Substance'] + r['name endings 2']) if r['Substance'].endswith('-') or r['Substance'].endswith(',') else r['Substance'], axis=1)

In [7]:
df

,Substance,Hcp,name endings,name endings 2
0,nan,Inor,nan,oxygen
1,nan,O,oxygen,O2
2,oxygen,1.2×10−5,O2,[7782-44-7]
3,O2,1.3 ×10−5,[7782-44-7],nan
4,[7782-44-7],1.3×10−5,nan,nan
...,...,...,...,...
23823,(methyltriethyl lead)[1762-28-3],NaN,[1762-28-3],tetraethyllead
23824,[1762-28-3],NaN,tetraethyllead,C8H20Pb
23825,tetraethyllead,1.3×10−5,C8H20Pb,[78-00-2]
23826,C8H20Pb,1.3×10−5,[78-00-2],NaN


In [8]:
# we can now drop the name endings and name endings 2 columns as we are done with them
df = df.drop(df.columns[[2, 3]], axis=1)

In [9]:
# NaN values in column 1 will not correspond to any Henry's law constants or IUPAC names so can be removed
df.dropna(inplace=True)

In [33]:
df[df['Substance'] == '1,2,3,4,6,7,8-heptachlorodibenzofuran']

,Substance,Hcp
18250,"1,2,3,4,6,7,8-heptachlorodibenzofuran",7.0 −1×10


In [10]:
df

,Substance,Hcp
0,nan,Inor
1,nan,O
2,oxygen,1.2×10−5
3,O2,1.3 ×10−5
4,[7782-44-7],1.3×10−5
...,...,...
23814,ethyltrimethylplumbane,2.8×10−5
23817,diethyldimethylplumbane,2.1 ×10−5
23821,triethylmethylplumbane,1.6×10−5
23825,tetraethyllead,1.3×10−5


In [11]:
# we now need to get rid of rows which have Substance value representing chemical formula and CAS IDs as these represent duplicates
# we know that the IUPAC names all start with a lowercase letter or a digit so we can filter by that (as opposed to an uppercase letter or square bracket as the above would)
df_filtered = df[df.Substance.str.contains('^[0-9a-z]')]

# stereoisomers such as E, Z, S, R, - are written as (E) so these will need to be accounted for as well
df_stereo = df[df.Substance.str.contains('^\([^a-z]+\)')]

# some names start with a bracket followed by a digit or lowercase - how to differentiate this to the 'Other Names'?
# well the IUPAC names will contain a hyphen after any starting bracket
df_brackets = df[df.Substance.str.contains('^\(\S+\)\-')]

# combine two filtered df together
df_clean = pd.concat([df_filtered, df_stereo, df_brackets])

In [12]:
df_clean

,Substance,Hcp
0,nan,Inor
1,nan,O
2,oxygen,1.2×10−5
5,nan,1.3×10−5
6,nan,1.3 10−5×
...,...,...
17262,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",1.7×10−1
20397,(bromomethyl)-benzene,1.4 10−3×
20444,(2-bromoethyl)-benzene,6.5×10−3
20939,"(2E)-N,N’-bis(2,4,6-tribromophenyl)-2-butenedi...",9.0 ×109


In [13]:
df_clean.Substance.duplicated().sum()

7184

In [14]:
# removing duplicates
df_clean = df_clean.drop_duplicates(subset=['Substance'])

In [15]:
# reset index
df_clean.reset_index(drop=True, inplace=True)
df_clean

,Substance,Hcp
0,nan,Inor
1,oxygen,1.2×10−5
2,ozone,1.0×10−4
3,hydrogen atom,2.6 ×10−6
4,hydrogen,7.8×10−6
...,...,...
4723,"(2,4-dichlorophenoxy)-acetic acid 2-ethylhexyl...",5.5×10−1
4724,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",1.7×10−1
4725,(bromomethyl)-benzene,1.4 10−3×
4726,(2-bromoethyl)-benzene,6.5×10−3


In [16]:
# one thing to note before dealing with the Hcp values is that some have a '>' in front of them - we will need to remove this in order to work with them
df_clean['Hcp'] = df_clean.apply(lambda r: (r['Hcp'].replace('>', '')) if r['Hcp'].startswith('>') else r['Hcp'], axis=1)

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# at the moment the Hcp values are a bit of a mess - we need to standardise them into a single format and then convert them into floats

# there are a few different situations to deal with - the first is the position of the x being in the wrong position
def correct_x_position(Hcp_value):
    if len(Hcp_value) <= 3:       # accounts for any values which aren't in standard form, e.g. 1.2
        Hcp_value = Hcp_value
    
    elif Hcp_value[3] != '×':
        Hcp_value = Hcp_value.replace('×', '').replace(' ','')    # any × characters at the end of the string are removed and then any whitespaces present as well
        Hcp_value = Hcp_value[0:3] + '×' + Hcp_value[3:]    # × characters placed in correct posittion before 10
    
    return Hcp_value
        
df_clean['Hcp'] = df_clean['Hcp'].apply(correct_x_position)

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df_clean

,Substance,Hcp
0,nan,Ino×r
1,oxygen,1.2×10−5
2,ozone,1.0×10−4
3,hydrogen atom,2.6×10−6
4,hydrogen,7.8×10−6
...,...,...
4723,"(2,4-dichlorophenoxy)-acetic acid 2-ethylhexyl...",5.5×10−1
4724,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",1.7×10−1
4725,(bromomethyl)-benzene,1.4×10−3
4726,(2-bromoethyl)-benzene,6.5×10−3


In [19]:
# next we convert the strings to floats to make it easier to work with
def standard_form(Hcp_value):
    if len(Hcp_value) <= 6:     # accounts for any values which aren't in standard form, e.g. 1.2
        Hcp_value = Hcp_value.replace('×10', 'e')
    
    elif Hcp_value[6] == '−':
        Hcp_value = Hcp_value.replace('−', '-')   # replace any − signs with the correct - sign
        Hcp_value = Hcp_value.replace('×10', 'e')    # converting values into scientific format that is understood by python
    
    elif Hcp_value[6] != '−':
        Hcp_value = Hcp_value[0:6] + '+' + Hcp_value[6:]   # insert + into any values without - sign to distinguish between 104 and 10^4
        Hcp_value = Hcp_value.replace('×10', 'e')    # converting values into scientific format that is understood by python
    
    return Hcp_value
    
df_clean['Hcp'] = df_clean['Hcp'].apply(standard_form)

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [20]:
df_clean

,Substance,Hcp
0,nan,Ino×r
1,oxygen,1.2e-5
2,ozone,1.0e-4
3,hydrogen atom,2.6e-6
4,hydrogen,7.8e-6
...,...,...
4723,"(2,4-dichlorophenoxy)-acetic acid 2-ethylhexyl...",5.5e-1
4724,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",1.7e-1
4725,(bromomethyl)-benzene,1.4e-3
4726,(2-bromoethyl)-benzene,6.5e-3


In [21]:
df_clean.loc[df_clean['Substance'] == '1,2,3,4,6,7,8-heptachlorodibenzofuran']

,Substance,Hcp
3108,"1,2,3,4,6,7,8-heptachlorodibenzofuran",7.0×−1+10


In [22]:
# there are some values which appear as just e rather than e-1 so we fix that here
df_clean['Hcp'] = [x + '-1' if x[-1] == 'e' else x for x in df_clean['Hcp']]

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
# converting string values to float 
df_clean['Hcp'] = df_clean['Hcp'].apply(pd.to_numeric, errors='coerce')

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
# clean up the numbers a bit
df_clean.Hcp = df_clean.Hcp.map('{:g}'.format)

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [25]:
# reset index
df_clean.reset_index(drop=True, inplace=True)
df_clean

,Substance,Hcp
0,nan,nan
1,oxygen,1.2e-05
2,ozone,0.0001
3,hydrogen atom,2.6e-06
4,hydrogen,7.8e-06
...,...,...
4723,"(2,4-dichlorophenoxy)-acetic acid 2-ethylhexyl...",0.55
4724,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",0.17
4725,(bromomethyl)-benzene,0.0014
4726,(2-bromoethyl)-benzene,0.0065


In [26]:
df_clean.loc[df_clean['Hcp'] == 'nan']

,Substance,Hcp
0,nan,nan
38,chlorine nitrate,nan
46,bromine nitrate,nan
55,sulfur trioxide,nan
100,"2,2-dimethylhexane",nan
136,"3,3,4-trimethylhexane",nan
200,"2,3,3,5-tetramethylhexane",nan
236,tricosane,nan
311,"1,3-butadiene",nan
359,"1,3-dimethylbenzene",nan


Looking back at the original dataset in the pdf we can see that these nan values correspond to uncertain values such as >4.9x10-4 for nitrosyl chloride or infinity for chlorine nitrate

In [27]:
# dropping these values from the dataframe
df_clean = df_clean[df_clean.Hcp != 'nan']

In [28]:
# reset index
df_clean.reset_index(drop=True, inplace=True)
df_clean

,Substance,Hcp
0,oxygen,1.2e-05
1,ozone,0.0001
2,hydrogen atom,2.6e-06
3,hydrogen,7.8e-06
4,deuterium,7.9e-06
...,...,...
4687,"(2,4-dichlorophenoxy)-acetic acid 2-ethylhexyl...",0.55
4688,"(2,4-dichlorophenoxy)-acetic acid,isooctyl ester",0.17
4689,(bromomethyl)-benzene,0.0014
4690,(2-bromoethyl)-benzene,0.0065


We are now very close to 4632 unique species

Looking towards the end of the dataframe we can see that some names are incomplete, they may have spilled over to the next row and therefore were deleted. We now go and correct those.

In [30]:
df_clean['Substance'] = df_clean.Substance.str.capitalize()

C:\Users\Alastair\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_clean.to_csv('cleaned_henry_dataset.csv', index=False)